In [1]:
library(lme4)
library(simr)
library(dplyr)
library(broom)
install.packages("sjPlot")
library(sjPlot)
install.packages("broom.mixed")
library(broom.mixed)
library(htmlTable)
library(dplyr)
library(stringr)
library(car)
install.packages("stargazer")
library(stargazer)
install.packages("lmerTest")
library(lmerTest)

Loading required package: Matrix


Attaching package: ‘simr’


The following object is masked from ‘package:lme4’:

    getData



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//Rtmpg6FyxD/downloaded_packages


Learn more about sjPlot with 'browseVignettes("sjPlot")'.




The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//Rtmpg6FyxD/downloaded_packages



Attaching package: ‘stringr’


The following object is masked from ‘package:simr’:

    fixed


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//Rtmpg6FyxD/downloaded_packages



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//Rtmpg6FyxD/downloaded_packages



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [2]:

file_path_total <- 'correct_wm_ltm_p.csv'
file_path_5 <- 'correct_wm_ltm_p_pilot5.csv'
file_path_6 <- 'correct_wm_ltm_p_pilot6.csv'



df_total <- read.csv(file_path_total) 
df5 <- read.csv(file_path_5) 
df6 <- read.csv(file_path_6) 


In [3]:
control <- lmerControl(optimizer = "bobyqa")

In [4]:
df <- df_total

In [5]:
run_ltm_models <- function(df) {
    


    ltm_rel <- glmer(recall_answ ~   p_values_v2_flipped_ltm + p_values_it_flipped_ltm +
                        + reliability_binary_z *  attend_binary_z  + tested_binary_z + it_pos_neg_ltm + v2_pos_neg_ltm + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))
     return(list(
    ltm_rel       = ltm_rel

  ))
}

In [6]:
results_old_stims <- run_ltm_models(df6)
results_new_stims <- run_ltm_models(df5)
results_total <- run_ltm_models(df_total)


ERROR: Error in eval(predvars, data, env): object 'p_values_v2_flipped_ltm' not found


In [ ]:
library(htmlTable)

extract_significance <- function(model_summary) {
  coef_table <- model_summary$coefficients
  var_names <- rownames(coef_table)
  
  # Get p-values
  if ("Pr(>|z|)" %in% colnames(coef_table)) {
    p_values <- coef_table[, "Pr(>|z|)"]
  } else if ("Pr(>|t|)" %in% colnames(coef_table)) {
    p_values <- coef_table[, "Pr(>|t|)"]
  } else {
    stop("Cannot find p-values in coefficients table")
  }
  
  # Calculate significance stars
  significance <- ifelse(is.na(p_values), "",
                        ifelse(p_values < 0.001, "***",
                              ifelse(p_values < 0.01, "**",
                                    ifelse(p_values < 0.05, "*",
                                          ifelse(p_values < 0.1, ".", "")))))
  
  return(data.frame(
    variable = var_names,
    p_value = p_values,
    significance = significance,
    stringsAsFactors = FALSE
  ))
}

compare_models <- function(model_list, model_names, 
                          include_p_values = TRUE,
                          only_significant = TRUE) {
  
  # Extract significance for all models
  sig_list <- lapply(model_list, extract_significance)
  names(sig_list) <- model_names
  
  # Get all variables
  all_vars <- unique(unlist(lapply(sig_list, function(x) x$variable)))
  
  # Create result dataframe
  result_df <- data.frame(Variable = all_vars, stringsAsFactors = FALSE)
  
  # Add columns for each model
  for (model_name in model_names) {
    model_sig <- sig_list[[model_name]]
    
    result_df[[model_name]] <- sapply(all_vars, function(var) {
      idx <- which(model_sig$variable == var)
      if (length(idx) == 0) return("")
      
      p_val <- model_sig$p_value[idx]
      stars <- model_sig$significance[idx]
      
      if (is.na(p_val)) return("")
      
      if (include_p_values) {
        if (stars != "") {
          return(paste0(stars, "<br>", sprintf("%.4f", p_val)))
        } else {
          return(sprintf("%.4f", p_val))
        }
      } else {
        return(stars)
      }
    })
  }
  
  # Filter to only significant if requested
  if (only_significant) {
    # Keep rows where at least one model has significance
    has_sig <- apply(result_df[, -1, drop = FALSE], 1, function(row) {
      any(grepl("[*.]", row))
    })
    result_df <- result_df[has_sig, , drop = FALSE]
  }
  
  # Remove completely empty rows
  non_empty <- apply(result_df[, -1, drop = FALSE], 1, function(row) any(row != ""))
  result_df <- result_df[non_empty, , drop = FALSE]
  
  # Create HTML table
  html_table <- htmlTable(
    result_df,
    caption = "Model Comparison<br><small>*** p&lt;0.001, ** p&lt;0.01, * p&lt;0.05, . p&lt;0.1</small>",
    rnames = FALSE,
    css.cell = "padding: 6px; border: 1px solid #ddd;",
    css.table = "border-collapse: collapse; width: 100%; font-family: Arial, sans-serif;",
    css.header = "background-color: #f2f2f2; font-weight: bold;"
  )
  
  return(html_table)
}


In [ ]:
ltm_rel_old <- summary(results_old_stims$ltm_rel)
ltm_rel_new <- summary(results_new_stims$ltm_rel)
ltm_rel_total <- summary(results_total$ltm_rel)

comparison_table_rel <- compare_models(
  model_list = list(ltm_rel_old, ltm_rel_new, ltm_rel_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,
  only_significant = TRUE
)

In [ ]:
IRdisplay::display_html(comparison_table_rel)


In [ ]:
ltm_rel_old

In [ ]:
ltm_rel_new

In [ ]:
# ltm_rel <- glmer(recall_answ ~   p_values_v2_flipped_ltm + p_values_it_flipped_ltm +
#                         + reliability_binary_z *  attend_binary_z  + tested_binary_z + it_pos_neg_ltm + v2_pos_neg_ltm + (1 | participant), 
#                       data = df_total, family = binomial,control = glmerControl(optimizer = "bobyqa",
#                                             optCtrl = list(maxfun = 500000),
#                                             calc.derivs = FALSE))


# summary(ltm_rel)

In [ ]:

betas <- fixef(ltm_rel)

# If you want more detailed information including standard errors, t-values, etc.
summary_model <- summary(ltm_rel)
coef_table <- summary_model$coefficients
pred_response <- predict(ltm_rel, type = "response")


intercept_logit <- fixef(ltm_rel)[1]  # or coef(summary(ltm_rel))[1,1]

mean_ltm_accuracy <- plogis(intercept_logit)  # or exp(intercept_logit)/(1+exp(intercept_logit))
print(intercept_logit)
print(paste("Mean LTM Accuracy:", round(mean_ltm_accuracy, 4)))

# Convert to data frame for easier handling
betas_df <- data.frame(
  term = names(betas),
  beta = betas,
  row.names = NULL
)

# If you want the full coefficient table with SE, z-values, p-values:
full_coef_df <- data.frame(
  term = rownames(coef_table),
  beta = coef_table[, "Estimate"],
  se = coef_table[, "Std. Error"],
  z_value = coef_table[, "z value"],
  p_value = coef_table[, "Pr(>|z|)"],
  intercept = mean_ltm_accuracy,
  pred_response = pred_response[,"x"],
  row.names = NULL
)

# Save to CSV
write.csv(betas_df, "model_betas.csv", row.names = FALSE)
# Or for the full table:
write.csv(full_coef_df, "model_coefficients_full.csv", row.names = FALSE)

# To specifically get the beta for p_values_it_flipped_ltm:
beta_it_flipped <- betas["p_values_it_flipped_ltm"]
print(paste("Beta for p_values_it_flipped_ltm:", beta_it_flipped))

In [ ]:
pred_response <- predict(ltm_rel, type = "response")
print(pred_response)

In [ ]:
ltm_rel <- glmer(recall_answ ~   p_values_it_flipped_ltm + (1 | participant), 
                      data = df_total, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))

In [ ]:
ltm_rel <- glmer(recall_answ ~   p_values_it_flipped_ltm + (1 | participant), 
                      data = df_total, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))

newdat <- data.frame(p_values_it_flipped_ltm = seq(-1.2, 1.2, length.out = 1000000))
preds <- predict(ltm_rel, newdata = newdat, type = "response", re.form = NA)
write.csv(preds, "preds_response_it.csv", row.names = FALSE)





In [ ]:
df_total$rt_ltm <- df_total$image_recall_response.rt
df_correct <- subset(df_total, recall_answ == 1)
df_correct$log_rt_ltm <- log(df_correct$rt_ltm)

cols_to_scale <- c("reliability_binary_z", "attend_binary_z", "tested_binary_z",
                   "it_ltm_z", "it_ltm_sq_z", "it_ltm_cube_z", "v2_ltm_z", "v2_ltm_sq_z","v2_ltm_cube_z", 
              "it_ltm_diff_z", "it_ltm_diff_sq_z","it_ltm_diff_cube_z",  "v2_ltm_diff_z", "v2_ltm_diff_sq_z","v2_ltm_diff_cube_z", 
                  "it_ltm_diff_sign_z", "it_ltm_diff_sign_sq_z","it_ltm_diff_sign_cube_z", "v2_ltm_diff_sign_z", "v2_ltm_diff_sign_sq_z", "v2_ltm_diff_sign_cube_z")

df_correct[paste0(cols_to_scale, "_new")] <- lapply(df_correct[cols_to_scale], scale)
    
    
model_rel <- lmer(log_rt_ltm ~ p_values_v2_flipped_ltm + p_values_it_flipped_ltm +
                        + reliability_binary_z_new *  attend_binary_z_new  + tested_binary_z_new + (1 | participant), 
                      data = df_correct, control = control)

summary(model_rel)

